In [4]:
import numpy as np
import librosa , librosa.display
import matplotlib.pyplot as plt
import os
import pandas as pd

In [5]:
import torch

#use gpu instead of cpu
# Check if MPS (Metal Performance Shaders) is available for GPU acceleration on Apple Silicon
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: mps


### Processing CSV File

In [6]:

csv_file = 'archive (1)/index.csv'
audio_dir = 'archive (1)/data'  

df = pd.read_csv(csv_file)
df

,audio_file,text,gender
0,D6x81u_1.mp3,خليني ابدا الفيديو ده وانا بقول ان من,male
1,6f5WmH_2.mp3,ااا ملاحظتي الشخصيه استنتجت ان الستات ما,male
2,abSOwC_3.mp3,بقاش عندها دم,male
3,qqiArB_4.mp3,وان الستات بداوا يستغلوا الرجاله وان,male
4,afzWCf_5.mp3,الستات دول,male
...,...,...,...
408054,0VO9mA_255.mp3,الامراض في كل,female
408055,22yVw5_256.mp3,[موسيقى],female
408056,OrpHs8_257.mp3,مكان,female
408057,HndKfM_258.mp3,[موسيقى],female


In [7]:
df.shape

(408059, 3)

In [8]:
rows=df.shape[0]

In [9]:
dataset = df[0:rows] 
dataset = dataset.reset_index()
dataset

,index,audio_file,text,gender
0,0,D6x81u_1.mp3,خليني ابدا الفيديو ده وانا بقول ان من,male
1,1,6f5WmH_2.mp3,ااا ملاحظتي الشخصيه استنتجت ان الستات ما,male
2,2,abSOwC_3.mp3,بقاش عندها دم,male
3,3,qqiArB_4.mp3,وان الستات بداوا يستغلوا الرجاله وان,male
4,4,afzWCf_5.mp3,الستات دول,male
...,...,...,...,...
408054,408054,0VO9mA_255.mp3,الامراض في كل,female
408055,408055,22yVw5_256.mp3,[موسيقى],female
408056,408056,OrpHs8_257.mp3,مكان,female
408057,408057,HndKfM_258.mp3,[موسيقى],female


### Processing Audio

In [10]:
import tqdm
import math

In [11]:
# Function to display audio 
import noisereduce as nr
from IPython.display import Audio
def displayAudio(file_path,f= False):
    audio_data, _ = librosa.load(audio_dir+file_path, sr = 22050)
    reduced_noise =  nr.reduce_noise(y=audio_data, sr=22050) if f else audio_data
    return Audio(reduced_noise, rate=22050)

In [12]:
N_ROWS = 50100 # NUMBER OF ROWS TAKEN FROM THE DATA
SHIFT = 0
MAX_TEXT_LEN = 70 # MAX LENGTH OF TRANSCRIPTION
MAX_SEQ_LEN = 70 # MAX LENGTH OF DECODER INPUT SEQUENCE
N_MELS = 128 # NUMBER OF MELS
SAMPLE_RATE = 16000 # NUMBER OF SAMPLES PER SECOND
HOP_LENGTH = 512 # THE STEP LENGTH OF THE SLIDING WINDOW , Commonly set to one-fourth of N_FFT
N_FFT = 2048 # NUMBER OF FFT POINTS (WINDOW SIZE) THIS CONTROLS THE RESOLUTION OF THE FREQUENCY DOMAIN ANALYSIS
CHUNK_LENGTH = 15 # 15 SECOND CHUNK
N_SAMPLES = SAMPLE_RATE * CHUNK_LENGTH 
N_FRAMES =  math.ceil(N_SAMPLES / HOP_LENGTH) 

In [13]:
def pad_or_trim(audio_data,padding=False):
    if len(audio_data) > N_SAMPLES:
        audio_data = audio_data[:N_SAMPLES]
    elif len(audio_data) < N_SAMPLES:
        if padding:
            audio_data = np.pad(audio_data, (0, N_SAMPLES - len(audio_data)), mode='constant')
        else:
            audio_data = np.pad(audio_data, (0, N_SAMPLES - len(audio_data)), mode='mean')
    return audio_data

In [ ]:
# Function to load audio and generate the mel spectrogram
def preprocess_audio(file_path):

    audio_data, _ = librosa.load(file_path, sr = 22050) #Loading audio
    
    audio_data = nr.reduce_noise(y=audio_data, sr=22050)

    duration = librosa.get_duration(y = audio_data , sr = 22050) #Get the duration

    modified_audio  = pad_or_trim(audio_data,padding = False) # Trim the audio if exceeds N_SAMPLES
    
    sgram = librosa.stft(y = modified_audio, n_fft = N_FFT, hop_length = HOP_LENGTH) # Applying Short Time Fourier Transform and returns 2D array
    
    sgram_mag, _ = librosa.magphase(sgram) # Seperates the spectrogram into magnitude and phase, we want magnitude only but phase is important for constructing the orginal signal 
    
    # Transforms the linear frequency spectrogram into Mel scale 
    mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=22050 , n_fft = N_FFT, hop_length = HOP_LENGTH ,n_mels = N_MELS)
    
    # Convert to DB scale 
    mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)
    
    # Delete the created variables
    del audio_data, modified_audio, sgram, mel_scale_sgram
    
    return mel_sgram , duration


processed_audios = []
transcriptions = []
max_duration = 0 
min_duration = 1000000
from tqdm import tqdm

for idx, row in tqdm(dataset.iterrows(), total=len(dataset)):
    
    # Get the audio file name and text transcription
    audio_filename = row['audio_file']
    transcription = row['text']
    
    # Build the full path to the audio file
    audio_path = os.path.join(audio_dir, audio_filename)
    
    # Preprocess the audio file
    try:
        processed_audio  , duration = preprocess_audio(audio_path)
        max_duration = max(max_duration,duration) 
        min_duration = min(min_duration,duration)
        processed_audios.append(processed_audio)
        transcriptions.append(transcription)
    except Exception as e:
        print(f"Error processing {audio_filename}: {e}")



  1%|          | 3073/408059 [01:27<2:03:49, 54.51it/s]/var/folders/c2/f9lh6rmd4q1648_pfl1636zw0000gn/T/ipykernel_77409/1278201152.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, _ = librosa.load(file_path, sr = 22050) #Loading audio
/Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  1%|          | 3086/408059 [01:28<2:02:36, 55.05it/s]

Error processing لشخصك ولأفكارك الـProgressive،": [Errno 2] No such file or directory: 'archive (1)/data/لشخصك ولأفكارك الـProgressive،"'


  8%|▊         | 32761/408059 [14:49<2:20:28, 44.53it/s]  /Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/noisereduce/spectralgate/nonstationary.py:70: RuntimeWarning: invalid value encountered in divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth
  8%|▊         | 32766/408059 [14:49<2:21:55, 44.07it/s]

Error processing 8VvErN_634.mp3: Audio buffer is not finite everywhere


 11%|█         | 44949/408059 [19:33<2:37:26, 38.44it/s]

...

Error processing pNuehK_2176.mp3: Audio buffer is not finite everywhere


 83%|████████▎ | 338390/408059 [5:48:25<29:57, 38.76it/s]  

Error processing ypZjew_595.mp3: Audio buffer is not finite everywhere


100%|██████████| 408059/408059 [6:14:19<00:00, 18.17it/s]


: 

In [ ]:
padded_audios = [( mel_spec.shape[-1],np.pad(mel_spec, ((0, 0), (0, N_FRAMES - mel_spec.shape[-1])))) for mel_spec in processed_audios]

del processed_audios 

In [ ]:
print(padded_audios[2][1].shape)
librosa.display.specshow(padded_audios[0][1], sr=SAMPLE_RATE, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')

### Transcriptions Processing

In [ ]:
from num2words import num2words
import re
# Function to replace numbers with words in a single sentence
def nlp_preprocessing(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace("\n", " ")

    # Replace the variants of 2lf
    sentence = re.sub(r'[إأآ]', 'ا', sentence)

    # Remove the punctuation and special characters
    sentence = re.sub(r'[^a-zA-Zء-ي\s\d]', '', sentence)  # Keep only Arabic, English letters, digits, and spaces

    pattern = r'[\u0617-\u061A\u064B-\u065F]'

    # Remove all the arabic special characters (تشكيل)
    sentence = re.sub(pattern, '', sentence)

    # Pattern to insert a space between Arabic and English (or vice versa)
    pattern = r'([a-zA-Z])([ء-ي])|([ء-ي])([a-zA-Z])'

    # Replace matches with the same characters separated by a space
    sentence = re.sub(pattern, r'\1\3 \2\4', sentence)

    # Replace multiple spaces with a single space
    sentence = re.sub(r'\s+', ' ', sentence)

    # Regular expression to find numbers in the sentence and replace them with words
    return re.sub(r'\d+', lambda x: num2words(int(x.group()), lang='ar'), sentence)

# Apply the function to each sentence in the array
modified_transcriptions = [nlp_preprocessing(sentence) for sentence in transcriptions]
#why when  running it once some patterns does not fully applied to the text?
modified_transcriptions = [nlp_preprocessing(sentence) for sentence in modified_transcriptions]



In [ ]:
import string

# Define the vocabulary: add lowercase letters, digits, and special tokens
special_tokens = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']  # Special tokens
english_characters = list(string.ascii_lowercase + ' ')  # List of English characters
arabic_characters = list("ابتثجحخدذرزسشصضطظعغفقكلمنهويئءىةؤ")  # List of Arabic characters

# Combine all characters
characters = english_characters + arabic_characters
vocab = special_tokens + characters  # Combine special tokens and characters

# Build dictionaries for character-to-index and index-to-character
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = {idx: char for idx, char in enumerate(vocab)}

vocab_size = len(vocab)
print(f"Vocabulary size: {vocab_size}")

print(vocab)

In [ ]:
def tokenize_text(text, char2idx, max_len = MAX_SEQ_LEN , start_token = True , end_token = True):
        
    tokens = [char2idx.get(char, char2idx['<UNK>']) for char in text]
    
    max_len -=  start_token + end_token # fot the start and end token

    # pad to a maximum length (for batching)
    if max_len is not None:
        text_len = len(tokens)
        if text_len < max_len:
            if end_token: tokens += [char2idx['<EOS>']]
            tokens += [char2idx['<PAD>']] * (max_len - text_len)  # Pad
        else:
            tokens = tokens[:max_len]  # Truncate if longer than max_len
            if end_token: tokens += [char2idx['<EOS>']]
            
    if start_token: tokens.insert(0,char2idx['<SOS>'])
    
    return tokens

# Example tokenization
sentence = "السلام عليكم"
tokens = tokenize_text(sentence, char2idx, max_len = MAX_SEQ_LEN)
print(tokens)

In [ ]:
import torch

In [ ]:

# Function to decode the tokenized text back to its original form 
def TextDecoder(sentence):
    out = ''
    for token in sentence:
        if isinstance(token, torch.Tensor):
            token = token.item()
        char = idx2char[token]
        if char == '<EOS>':
            return out
        if not (char in special_tokens):
            out += char
    return out
TextDecoder(tokens)        

In [ ]:
tokenized_transcriptions_2 = modified_transcriptions

In [ ]:
len(tokenized_transcriptions_2)

### Text to semantic model